## ACS PUMS: recent adult datasets

How to use folktables please refer to https://github.com/socialfoundations/folktables.git
- e.g. the defination of adult_filter can be found at https://github.com/socialfoundations/folktables/blob/main/folktables/acs.py

<span style="color:red">Quick note about the tool:</span>
The original paper (https://arxiv.org/pdf/2108.04884v3) provides detailed descriptions of datasets like ACSIncome and ACSEmployment, including their specific features and filter conditions.

**The ```ACSIncome``` function:**
1. comes with default configurations:
- Features(float): AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P
- Labels(boolean): PINCP

2. ```adult_filter``` criteria:
- AGEP (Age): Must be greater than 16
- PINCP (Total person’s income): Must be greater than 100
- WKHP (Usual hours worked per week past 12 months): Must be greater than 0
- PWGTP (Person weight (relevant for re-weighting dataset to represent the general US population most accurately)): Must be greater than or equal to 1

**The ```ACSEmployment``` function:**
1. comes with default configurations:
- Features(float):'AGEP','SCHL','MAR','SEX','DIS','ESP','MIG','CIT','MIL','ANC','NATIVITY','RELP','DEAR','DEYE','DREM','RAC1P','GCL'
- Labels(boolean):'ESR'
- **The label should be <span style="color:red">'ESR'</span>**. According to the documentation, the feature ‘NWLA’ (On layoff from work) is UNEDITED – see ‘Employment Status Recode’ (ESR).
2. ```employment_filter``` criteria:
- AGEP (Age) must be greater than 16 and less than 90.
- PWGTP (Person weight) must be greater than or equal to 1.

In both datasets, **we dropped the ‘RELP’ feature** because it is not available in the 2023 dataset, and we want to ensure that all features are consistent across years. See code below.
## Data Preprocessing

### Assign OCCP Top-Level Category

In [1]:
import requests
import pandas as pd

# API URL of OCCP "Occupation"
url_2014 = "https://api.census.gov/data/2014/acs/acs1/pums/variables/OCCP.json"
url_2023 = "https://api.census.gov/data/2023/acs/acs1/pums/variables/OCCP.json"
response_2014 = requests.get(url_2014)
response_2023 = requests.get(url_2023)
data_2014 = response_2014.json()
data_2023 = response_2023.json()

# OCCP
print(data_2014['label'])
print(data_2023['label'])

dict_2014 = data_2014['values']['item']
dict_2023 = data_2023['values']['item']
# Merging the two dictionaries and cover
# Keys present in both 2014 and 2023 are updated with 2023 values.
values_dict = {**dict_2014, **dict_2023}

df_OCCP = pd.DataFrame.from_dict(values_dict, orient='index', columns=['explanation'])
df_OCCP = df_OCCP.reset_index().rename(columns={'index': 'code'})
df_OCCP = df_OCCP[df_OCCP['code'] != 'N']
df_OCCP['code'] = df_OCCP['code'].astype(float)
# Extracting the first word as the category if it exists, otherwise set it to None
df_OCCP['category_prefix'] = df_OCCP['explanation'].str.extract(r'^([^-\s]+)-')

# Full Name Mapping
prefix_dict = {
    "MGR": "Manager",
    "OFF": "Office and Administrative Support",
    "EAT": "Food Preparation and Serving Related",
    "PRD": "Production",
    "TRN": "Transportation and Material Moving",
    "CMS": "Community and Social Service",
    "BUS": "Business and Financial Operations",
    "ENG": "Engineers",
    "SCI": "Scientists",
    "MED": "Healthcare Practitioners and Technical",
    "HLS": "Healthcare Support",
    "PRT": "Protective Service",
    "EDU": "Education, Training, and Library",
    "ENT": "Arts, Design, Entertainment, Sports, and Media",
    "LGL": "Legal",
    "SAL": "Sales and Related",
    "PRS": "Personal Care and Service",
    "CLN": "Building and Grounds Cleaning and Maintenance",
    "CON": "Construction and Extraction",
    "RPR": "Installation, Maintenance, and Repair",
    "FFF": "Farming, Fishing, and Forestry",
    "CMM": "Computer and Mathematical",
    "FIN": "Financial Specialists",
    "EXT": "Extraction Workers",
    "MIL": "Military Specific"
}

prefix_code_dict = {k: i for i, k in enumerate(prefix_dict.keys())}
df_OCCP['category_code'] = df_OCCP['category_prefix'].map(prefix_code_dict)
df_OCCP['category_name'] = df_OCCP['category_prefix'].map(prefix_dict)

print(df_OCCP.head())
print("Size of df_OCCP: ", df_OCCP.shape)

Occupation recode for 2012 and later based on 2010 OCC codes
Occupation recode for 2018 and later based on 2018 OCC codes
     code                                explanation category_prefix  \
0  2000.0                             CMS-COUNSELORS             CMS   
1  3160.0                    MED-Physical Therapists             MED   
2  3820.0  PRT-Detectives And Criminal Investigators             PRT   
3  5200.0                       OFF-BROKERAGE CLERKS             OFF   
4  5220.0   OFF-Court, Municipal, And License Clerks             OFF   

   category_code                           category_name  
0            5.0            Community and Social Service  
1            9.0  Healthcare Practitioners and Technical  
2           11.0                      Protective Service  
3            1.0       Office and Administrative Support  
4            1.0       Office and Administrative Support  
Size of df_OCCP:  (631, 5)


In [2]:
df_OCCP.info()

<class 'pandas.core.frame.DataFrame'>
Index: 631 entries, 0 to 631
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   code             631 non-null    float64
 1   explanation      631 non-null    object 
 2   category_prefix  629 non-null    object 
 3   category_code    629 non-null    float64
 4   category_name    629 non-null    object 
dtypes: float64(2), object(3)
memory usage: 29.6+ KB


In [3]:
df_OCCP[['category_prefix', 'category_code']].drop_duplicates()

,category_prefix,category_code
0,CMS,5.0
1,MED,9.0
2,PRT,11.0
3,OFF,1.0
5,TRN,4.0
7,CON,18.0
8,NaN,NaN
9,FIN,22.0
10,HLS,10.0
11,MIL,24.0


### Require Data

In [4]:
from folktables import ACSDataSource, ACSIncome, ACSEmployment, BasicProblem, adult_filter, employment_filter
import numpy as np

# Dropping 'RELP' since it is not available in 2023 dataset
ACSIncomeNew = BasicProblem( 
    features=[
        'AGEP',
        'COW',
        'SCHL',
        'MAR',
        'OCCP',
        'POBP',
        'WKHP',
        'SEX',
        'RAC1P',
    ],
    target='PINCP',
    target_transform=lambda x: x > 50000,    
    group='RAC1P',
    preprocess=adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

ACSEmploymentNew = BasicProblem(
    features=[
        'AGEP',
        'SCHL',
        'MAR',
        'SEX',
        'DIS',
        'ESP',
        'MIG',
        'CIT',
        'MIL',
        'ANC',
        'NATIVITY',
        'DEAR',
        'DEYE',
        'DREM',
        'RAC1P',
        'GCL',
    ],
    target="ESR",
    target_transform=lambda x: x == 1,
    group='RAC1P',
    preprocess=employment_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

data_source_2014 = ACSDataSource(survey_year='2014', horizon='1-Year', survey='person')
ca_data_2014 = data_source_2014.get_data(states=["CA"], download=True) # California

data_source_2023 = ACSDataSource(survey_year='2023', horizon='1-Year', survey='person')
ca_data_2023 = data_source_2023.get_data(states=["CA"], download=True) # California

In [5]:
# Income Data
ca_features_2023, ca_labels_2023, _ = ACSIncomeNew.df_to_pandas(ca_data_2023)
ca_features_2014, ca_labels_2014, _ = ACSIncomeNew.df_to_pandas(ca_data_2014)

# Employment Data
ca_features_2014_em, ca_labels_2014_em, _ = ACSEmploymentNew.df_to_pandas(ca_data_2014)
ca_features_2023_em, ca_labels_2023_em, _ = ACSEmploymentNew.df_to_pandas(ca_data_2023)

We can think about whether to use all features together for both prediction tasks (income and employment), or just follow Ding et al.’s approach and use the features they grouped separately for each task.

#### Mapping Occupation Codes to Occupation Categories

In [6]:
ca_features_2014_merged = ca_features_2014.merge(df_OCCP[['code', 'category_code']], 
                            left_on='OCCP', right_on='code', how='left').drop(columns=['code']).rename(columns={'category_code': 'OCCP_CA'})
ca_features_2023_merged = ca_features_2023.merge(df_OCCP[['code', 'category_code']],
                            left_on='OCCP', right_on='code', how='left').drop(columns=['code']).rename(columns={'category_code': 'OCCP_CA'})

In [7]:
ca_features_2014_merged

,AGEP,COW,SCHL,MAR,OCCP,POBP,WKHP,SEX,RAC1P,OCCP_CA
0,36.0,1.0,16.0,5.0,9610.0,22.0,35.0,1.0,1.0,4.0
1,47.0,1.0,20.0,3.0,5700.0,39.0,60.0,2.0,1.0,1.0
2,20.0,1.0,19.0,5.0,4840.0,6.0,40.0,1.0,1.0,15.0
3,31.0,1.0,19.0,1.0,5120.0,6.0,45.0,2.0,1.0,1.0
4,31.0,5.0,19.0,1.0,5550.0,6.0,40.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
183936,63.0,1.0,19.0,3.0,4720.0,313.0,40.0,2.0,8.0,15.0
183937,38.0,3.0,20.0,5.0,3320.0,6.0,40.0,1.0,8.0,9.0
183938,40.0,1.0,21.0,1.0,6440.0,6.0,30.0,2.0,1.0,18.0
183939,40.0,3.0,20.0,1.0,3740.0,6.0,56.0,1.0,1.0,11.0


In [8]:
ca_features_2023_merged

,AGEP,COW,SCHL,MAR,OCCP,POBP,WKHP,SEX,RAC1P,OCCP_CA
0,19.0,1.0,19.0,5.0,4055.0,6.0,16.0,2.0,8.0,2.0
1,23.0,1.0,19.0,5.0,2850.0,6.0,15.0,1.0,1.0,13.0
2,23.0,1.0,20.0,5.0,2850.0,6.0,15.0,1.0,1.0,13.0
3,28.0,1.0,19.0,5.0,4760.0,6.0,40.0,1.0,9.0,15.0
4,20.0,1.0,19.0,5.0,4640.0,6.0,20.0,2.0,2.0,16.0
...,...,...,...,...,...,...,...,...,...,...
203273,43.0,1.0,22.0,5.0,51.0,6.0,40.0,2.0,1.0,0.0
203274,40.0,1.0,21.0,1.0,3255.0,6.0,24.0,2.0,1.0,9.0
203275,43.0,1.0,22.0,1.0,1320.0,6.0,50.0,1.0,1.0,7.0
203276,33.0,1.0,21.0,3.0,1050.0,508.0,40.0,2.0,6.0,21.0


In [9]:
result = (ca_features_2014_merged.groupby('OCCP_CA').size().reset_index(name='Count')
          .sort_values('Count', ascending=False).reset_index(drop=True))
print(len(ca_features_2014_merged))
print(result)

183941
    OCCP_CA  Count
0       1.0  23420
1      15.0  19227
2       0.0  19144
3      12.0  11307
4       4.0  10452
5       2.0   9669
6       3.0   9438
7       9.0   9259
8      16.0   8458
9      18.0   7948
10     17.0   7133
11     21.0   5916
12     13.0   5711
13      6.0   5241
14     19.0   4794
15     22.0   4364
16      7.0   3988
17     11.0   3781
18     10.0   3322
19     20.0   3144
20      5.0   2951
21     14.0   2382
22      8.0   2101
23     24.0    660
24     23.0    131


In [10]:
result = (ca_features_2023_merged.groupby('OCCP_CA').size().reset_index(name='Count')
          .sort_values('Count', ascending=False).reset_index(drop=True))
print(len(ca_features_2023_merged))
print(result)

203278
    OCCP_CA  Count
0       0.0  23611
1       1.0  20052
2      15.0  17271
3       4.0  14141
4      12.0  12942
5       9.0  11375
6       2.0  10468
7      21.0   9386
8       3.0   8355
9      18.0   8125
10      6.0   8102
11     10.0   7579
12     13.0   6754
13     17.0   6660
14      7.0   5730
15     16.0   5656
16     19.0   4999
17     22.0   4489
18     11.0   3946
19      5.0   3608
20      8.0   3354
21     20.0   2747
22     14.0   2722
23     24.0   1102
24     23.0    104


In [11]:
ca_labels_2014 # True 1 False 0, 1 = > 50K, 0 = <= 50K

,PINCP
0,False
1,False
2,False
3,True
4,False
...,...
183936,False
183937,True
183938,False
183939,True


### Documentation, For later encoding categorical features or matching analysis.


<span style="color:red">Tip: Edit the text after the last "/" (like occp, cow, mar) to get the 2023 docs.</span>

In [12]:
import requests
import pandas as pd

# API URL of OCCP "Occupation"
url = "https://api.census.gov/data/2023/acs/acs1/pums/variables/OCCP.json"
response = requests.get(url)
data = response.json()

# OCCP
print(data['label'])
values_dict = data['values']['item']
df_OCCP = pd.DataFrame.from_dict(values_dict, orient='index', columns=['Explanation'])
df_OCCP = df_OCCP.reset_index().rename(columns={'index': 'Code'})
print(df_OCCP.head())
print("Size of df_OCCP: ", df_OCCP.shape)

Occupation recode for 2018 and later based on 2018 OCC codes
   Code                                        Explanation
0  2060                   CMS-Religious Workers, All Other
1  4130  EAT-Dining Room And Cafeteria Attendants And B...
2  5340                            OFF-New Accounts Clerks
3  8610      PRD-Stationary Engineers And Boiler Operators
4  4020                                          EAT-Cooks
Size of df_OCCP:  (531, 2)


In [13]:
import requests
import pandas as pd

# API URL of COW "Class of worker"
url = "https://api.census.gov/data/2023/acs/acs1/pums/variables/COW.json"
response = requests.get(url)
data = response.json()

# df_COW
print(data['label'])
values_dict = data['values']['item']
df_COW = pd.DataFrame.from_dict(values_dict, orient='index', columns=['Explanation'])
df_COW = df_COW.reset_index().rename(columns={'index': 'Code'})
print(df_COW.head())
print("Size of df_COW: ", df_COW.shape)

Class of worker
  Code                                        Explanation
0    9  Unemployed and last worked 5 years ago or earl...
1    7  Self-employed in own incorporated business, pr...
2    3     Local government employee (city, county, etc.)
3    8     Working without pay in family business or farm
4    5                        Federal government employee
Size of df_COW:  (10, 2)


In [14]:
import requests
import pandas as pd

# API URL of MAR "Marital status"
url = "https://api.census.gov/data/2023/acs/acs1/pums/variables/MAR.json"
response = requests.get(url)
data = response.json()

# df_COW
print(data['label'])
values_dict = data['values']['item']
df_MAR= pd.DataFrame.from_dict(values_dict, orient='index', columns=['Explanation'])
df_MAR = df_MAR.reset_index().rename(columns={'index': 'Code'})
print(df_MAR.head())
print("Size of df_MAR: ", df_MAR.shape)

Marital status
  Code                          Explanation
0    1                              Married
1    5  Never married or under 15 years old
2    3                             Divorced
3    2                              Widowed
4    4                            Separated
Size of df_MAR:  (5, 2)
